In [ ]:
import pandas as pd
import gensim
from gensim.models import KeyedVectors
from stellargraph.data import EdgeSplitter
import networkx as nx
from sklearn.linear_model import LogisticRegressionCV
model = gensim.models.Word2Vec.load('../lm-vol/2_18_100_epochs_word2vec.model')
wv = KeyedVectors.load('../lm-vol/2_18_100_epochs_word2vec.wordvectors', mmap='r')
G = nx.read_graphml('../lm-vol/LANL_train.graphml')
def operator_hadamard(u, v):
    return u * v
node_embeddings = (wv.vectors) 
# Define an edge splitter on the original graph:
edge_splitter_test = EdgeSplitter(G)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from graph, and obtain the
# reduced graph graph_test with the sampled links removed:
graph_test, edges_train, labels_train = edge_splitter_test.train_test_split(
    p=0.99, method="global"
)
training_edges = [operator_hadamard(wv[edge[0]], wv[edge[1]]) for edge in edges_train]
clf = LogisticRegressionCV(cv=5, max_iter=1000, random_state=0).fit(training_edges, labels_train)
clf.score(training_edges, labels_train)
graph_structure = ('source', 'source_computer', 'destination')
inconclusive = 0
n = 0
chunk = pd.read_csv('../lm-vol/redteam_complete.csv')
#     display(chunk)
chunk_invalids = pd.DataFrame()
client_to_ip_pred = []
ip_to_service_pred = []
for index, row in chunk.iterrows():
    try: 
        client_embedding = wv[(row[graph_structure[0]])]
        ip_embedding = wv[(row[graph_structure[1]])]
        service_embedding = wv[(row[graph_structure[2]])]
        client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
        ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
        client_to_ip = clf.predict_proba(client_to_ip_embedding)
        ip_to_service = clf.predict_proba(ip_to_service_embedding)
        client_to_ip_pred.append(client_to_ip[0][1])
        ip_to_service_pred.append(ip_to_service[0][1])
    except: 
        client_to_ip_pred.append(None)
        ip_to_service_pred.append(None)
        inconclusive += 1
chunk['client_to_ip_pred'] = client_to_ip_pred
chunk['ip_to_service_pred'] = ip_to_service_pred
chunk_anomalies = chunk[(chunk["ip_to_service_pred"].astype(float) <= 0.1) | (chunk["client_to_ip_pred"].astype(float) <= 0.1)]
chunk_invalids = chunk[(chunk["ip_to_service_pred"] == None) & (chunk["client_to_ip_pred"] == None)]
display(chunk_anomalies)

** Sampled 630767 positive and 630767 negative edges. **


In [ ]:
display(chunk)

In [9]:
import pickle
# Save to file in the current working directory
pkl_filename = "../lm-vol/2_18_lr.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [16]:
import pandas as pd
with open('../lm-vol/2_18_epochs_lr.pkl', 'rb') as file:
    clf = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '../lm-vol/2_18_epochs_lr.pkl'

In [17]:
graph_structure = ('source', 'source_computer', 'destination')
inconclusive = 0
n = 0
chunk = pd.read_csv('../lm-vol/redteam_complete.csv')
#     display(chunk)
chunk_invalids = pd.DataFrame()
client_to_ip_pred = []
ip_to_service_pred = []
for index, row in chunk.iterrows():
    try: 
        client_embedding = wv[(row[graph_structure[0]])]
        ip_embedding = wv[(row[graph_structure[1]])]
        service_embedding = wv[(row[graph_structure[2]])]
        client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
        ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
        client_to_ip = clf.predict_proba(client_to_ip_embedding)
        ip_to_service = clf.predict_proba(ip_to_service_embedding)
        client_to_ip_pred.append(client_to_ip[0][1])
        ip_to_service_pred.append(ip_to_service[0][1])
    except: 
        client_to_ip_pred.append(None)
        ip_to_service_pred.append(None)
        inconclusive += 1
chunk['client_to_ip_pred'] = client_to_ip_pred
chunk['ip_to_service_pred'] = ip_to_service_pred
chunk_anomalies = chunk[(chunk["ip_to_service_pred"].astype(float) <= 0.1) | (chunk["client_to_ip_pred"].astype(float) <= 0.1)]
chunk_invalids = chunk[(chunk["ip_to_service_pred"] == None) & (chunk["client_to_ip_pred"] == None)]
display(chunk_anomalies)

,Unnamed: 0,time,source,source_computer,destination_computer,destination,authentication_type,logon_type,authentication_orientation,Success,client_to_ip_pred,ip_to_service_pred
3,3,151993,U6115@DOM1,C17693,C1173,U6115@DOM1,NTLM,Network,LogOn,Success,0.044621,0.044621
50,50,725488,U1653@DOM1,C17693,C395,U1653@DOM1,NTLM,Network,LogOn,Success,0.000619,0.000619
51,51,725589,U1653@DOM1,C17693,C2669,U1653@DOM1,NTLM,Network,LogOn,Success,0.000619,0.000619
63,63,727131,U212@DOM1,C17693,C126,U212@DOM1,NTLM,Network,LogOn,Success,0.017722,0.017722
64,64,727419,U212@DOM1,C17693,C126,U212@DOM1,NTLM,Network,LogOn,Success,0.017722,0.017722
...,...,...,...,...,...,...,...,...,...,...,...,...
703,703,2305554,U4448@DOM1,C17693,C1555,U4448@DOM1,NTLM,Network,LogOn,Success,0.071163,0.071163
705,705,2306980,U6572@DOM1,C17693,C90,U6572@DOM1,NTLM,Network,LogOn,Success,0.008331,0.008331
707,707,2364275,U4978@DOM1,C17693,C1448,U4978@DOM1,NTLM,Network,LogOn,Success,0.009493,0.009493
716,716,2461523,U3718@DOM1,C17693,C9692,U3718@DOM1,NTLM,Network,LogOn,Success,0.000373,0.000373


In [18]:
graph_structure = (1, 3, 2)
client_to_ip_pred = []
ip_to_service_pred = []
inconclusive = 0
for chunk in pd.read_csv('../lm-vol/LANL_test.csv', header=None, chunksize=1000000):
#     display(chunk)
    for index, row in chunk.iterrows():
        try: 
            client_embedding = wv[(row[graph_structure[0]])]
            ip_embedding = wv[(row[graph_structure[1]])]
            service_embedding = wv[(row[graph_structure[2]])]
            client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
            ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
            client_to_ip = clf.predict_proba(client_to_ip_embedding)
            ip_to_service = clf.predict_proba(ip_to_service_embedding)
            client_to_ip_pred.append(client_to_ip[0][1])
            ip_to_service_pred.append(ip_to_service[0][1])
        except: 
            client_to_ip_pred.append(None)
            ip_to_service_pred.append(None)
            inconclusive += 1
    chunk['client_to_ip_pred'] = client_to_ip_pred
    chunk['ip_to_service_pred'] = ip_to_service_pred
    break
anomalous_edges = len(chunk[(chunk["ip_to_service_pred"].astype(float) <= 0.1) | (chunk["client_to_ip_pred"].astype(float) <= 0.1)])
normal_edges = len(chunk[(chunk["ip_to_service_pred"].astype(float) > 0.1) & (chunk["client_to_ip_pred"].astype(float) > 0.1)])
print(anomalous_edges/(normal_edges+anomalous_edges))
print(anomalous_edges)
print(normal_edges)

0.10485086020052158
104735
894160


In [20]:
experiment_names = ['2_18_500_epochs']

def operator_hadamard(u, v):
        return u * v

for name in experiment_names:
    with open(f'../lm-vol/{name}_lr.pkl', 'rb') as file:
        clf = pickle.load(file)
    model = gensim.models.Word2Vec.load(f'../lm-vol/{name}_word2vec.model')
    wv = KeyedVectors.load(f'../lm-vol/{name}_word2vec.wordvectors', mmap='r')
    graph_structure = ('source', 'source_computer', 'destination')
    inconclusive = 0
    n = 0
    red = pd.read_csv('../lm-vol/redteam_complete.csv')
    client_to_ip_pred = []
    ip_to_service_pred = []
    for index, row in red.iterrows():
        try: 
            client_embedding = wv[(row[graph_structure[0]])]
            ip_embedding = wv[(row[graph_structure[1]])]
            service_embedding = wv[(row[graph_structure[2]])]
            client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
            ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
            client_to_ip = clf.predict_proba(client_to_ip_embedding)
            ip_to_service = clf.predict_proba(ip_to_service_embedding)
            client_to_ip_pred.append(client_to_ip[0][1])
            ip_to_service_pred.append(ip_to_service[0][1])
        except: 
            client_to_ip_pred.append(None)
            ip_to_service_pred.append(None)
            inconclusive += 1
    red['client_to_ip_pred'] = client_to_ip_pred
    red['ip_to_service_pred'] = ip_to_service_pred
    red_anomalies = red[(red["ip_to_service_pred"].astype(float) <= 0.1) | (red["client_to_ip_pred"].astype(float) <= 0.1)]
    red_invalids = red[(red["ip_to_service_pred"] == None) & (red["client_to_ip_pred"] == None)]
    display(red_anomalies)
    
    test_G = nx.read_graphml('../lm-vol/LANL_test.graphml')
    test = []
    # max = 100
    for index, edge in enumerate(test_G.edges):
    #     if index > max:
    #         break
        try:
            src = wv[edge[0]]
            dest = wv[edge[1]]
            src_to_dest = [operator_hadamard(src, dest)]
            prob = clf.predict_proba(src_to_dest)
            test.append(prob[0][1])
        except:
            test.append(None)
    anomalies = 0
    for prob in test:
        if prob is not None and prob <= 0.1:
            anomalies+=1
    print(anomalies)
#     graph_structure = (1, 3, 2)
#     client_to_ip_pred = []
#     ip_to_service_pred = []
#     inconclusive = 0
#     for chunk in pd.read_csv('../lm-vol/LANL_test.csv', header=None, chunksize=1000000):
#     #     display(chunk)
#         for index, row in chunk.iterrows():
#             try: 
#                 client_embedding = wv[(row[graph_structure[0]])]
#                 ip_embedding = wv[(row[graph_structure[1]])]
#                 service_embedding = wv[(row[graph_structure[2]])]
#                 client_to_ip_embedding = [operator_hadamard(client_embedding, ip_embedding)]
#                 ip_to_service_embedding = [operator_hadamard(ip_embedding, service_embedding)]
#                 client_to_ip = clf.predict_proba(client_to_ip_embedding)
#                 ip_to_service = clf.predict_proba(ip_to_service_embedding)
#                 client_to_ip_pred.append(client_to_ip[0][1])
#                 ip_to_service_pred.append(ip_to_service[0][1])
#             except: 
#                 client_to_ip_pred.append(None)
#                 ip_to_service_pred.append(None)
#                 inconclusive += 1
#         chunk['client_to_ip_pred'] = client_to_ip_pred
#         chunk['ip_to_service_pred'] = ip_to_service_pred
#         chunk_anomalies = chunk[(chunk["ip_to_service_pred"].astype(float) <= 0.1) | (chunk["client_to_ip_pred"].astype(float) <= 0.1)]
#         chunk_invalids = chunk[(chunk["ip_to_service_pred"] == None) & (chunk["client_to_ip_pred"] == None)]
#         display(chunk_anomalies)
#         break
    break

,Unnamed: 0,time,source,source_computer,destination_computer,destination,authentication_type,logon_type,authentication_orientation,Success,client_to_ip_pred,ip_to_service_pred
3,3,151993,U6115@DOM1,C17693,C1173,U6115@DOM1,NTLM,Network,LogOn,Success,0.044621,0.044621
50,50,725488,U1653@DOM1,C17693,C395,U1653@DOM1,NTLM,Network,LogOn,Success,0.000619,0.000619
51,51,725589,U1653@DOM1,C17693,C2669,U1653@DOM1,NTLM,Network,LogOn,Success,0.000619,0.000619
63,63,727131,U212@DOM1,C17693,C126,U212@DOM1,NTLM,Network,LogOn,Success,0.017722,0.017722
64,64,727419,U212@DOM1,C17693,C126,U212@DOM1,NTLM,Network,LogOn,Success,0.017722,0.017722
...,...,...,...,...,...,...,...,...,...,...,...,...
703,703,2305554,U4448@DOM1,C17693,C1555,U4448@DOM1,NTLM,Network,LogOn,Success,0.071163,0.071163
705,705,2306980,U6572@DOM1,C17693,C90,U6572@DOM1,NTLM,Network,LogOn,Success,0.008331,0.008331
707,707,2364275,U4978@DOM1,C17693,C1448,U4978@DOM1,NTLM,Network,LogOn,Success,0.009493,0.009493
716,716,2461523,U3718@DOM1,C17693,C9692,U3718@DOM1,NTLM,Network,LogOn,Success,0.000373,0.000373


18350
